In [1]:
#####start from train_FE.pkl
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import gc

label = pd.read_pickle('train_Basic.pkl')
train=pd.read_pickle('train_FE.pkl')
test=pd.read_pickle('test_FE.pkl')

y = label['isFraud']
X = train

/Users/ruiqianyang/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [ ]:
###############################################################
X['addr1'].fillna(0, inplace=True)
#X['addr2_dic'].fillna(3, inplace=True)
#X['addr2'].fillna(96, inplace=True)
#X['D5'].fillna(150, inplace=True)
X['D2'].fillna(10, inplace=True)
X['D11'].fillna(10, inplace=True)
X['D15'].fillna(376, inplace=True)

#X['V1'].fillna(X['V1'].mode()[0], inplace=True)          
#X['V2'].fillna(3, inplace=True)

#X['V263'].fillna(X['V263'].mode()[0], inplace=True)
#X['V256'].fillna(X['V256'].mode()[0], inplace=True)

X['card2'].fillna(502, inplace=True)
###X['card3'].fillna(150, inplace=True)
X['card4_dic'].fillna(X['card4_dic'].mode()[0], inplace=True)
                   
X['M4_dic'].fillna(2, inplace=True)
X['M5_dic'].fillna(3, inplace=True)  #24:18:78
X['M6_dic'].fillna(13, inplace=True)  #24:18:78

#X['cross'].fillna(2.6, inplace=True)

#X['id_19'].fillna(444, inplace=True)
#X['id_20'].fillna(266, inplace=True)
#X['id_17'].fillna(133, inplace=True)
X['P_emaildomain_dic'].fillna(0, inplace=True)
X['R_emaildomain_dic'].fillna(0, inplace=True)

#sep 04,2019 add fill na:
X['V13'].fillna(X['V13'].mode()[0], inplace=True)
X['V145'].fillna(0, inplace=True)  #V145 has more than 80% missing
X['V19'].fillna(0, inplace=True)
X['V36'].fillna(0, inplace=True)
X['V64'].fillna(0, inplace=True)
X['V70'].fillna(0, inplace=True)
X['V80'].fillna(0, inplace=True)
X['V94'].fillna(0, inplace=True)
# X['V143'].fillna(0, inplace=True)
# X['V150'].fillna(0, inplace=True)
# X['V152'].fillna(0, inplace=True)
X['V158'].fillna(0, inplace=True)##
# X['V163'].fillna(0, inplace=True)
# X['V165'].fillna(0, inplace=True)
# X['V177'].fillna(0, inplace=True)
# X['V204'].fillna(0, inplace=True)
# X['V207'].fillna(0, inplace=True)
# X['V209'].fillna(0, inplace=True) ##
# X['V221'].fillna(0, inplace=True)
# X['V222'].fillna(0, inplace=True)
# X['V266'].fillna(0, inplace=True)
# X['V267'].fillna(0, inplace=True)
# X['V274'].fillna(0, inplace=True)
# X['V275'].fillna(0, inplace=True)
X['V279'].fillna(0, inplace=True)
X['V283'].fillna(0, inplace=True)
##X['id_30_count'].fillna(0, inplace=True)

In [ ]:
#start RFECV
from sklearn.model_selection import KFold,GroupKFold
from sklearn.feature_selection import RFECV

X_1=X.copy()


for col_1 in X_1.columns:
    if X_1[col_1].dtype == 'object':
        print(col_1)
        le = LabelEncoder()
        le.fit(list(train[col_1].astype(str).values) + 
               list(test[col_1].astype(str).values))
        X_1[col_1] = le.transform(list(X_1[col_1].astype(str).values)
                             )

X_1 = X_1.fillna(-999)
X_1[np.isinf(X_1.values)==True] = 999
#X_1['DeviceInfo']=X['DeviceInfo'].cat.add_categories(-999).fillna(-999)
params1={'objective':'binary',
        'boosting_type':'gbdt',
        'metric':'auc',
        'n_jobs':-1,
        'learning_rate':0.05,
        'num_leaves': 2**8,
        'max_depth':25,
        'tree_learner':'serial',
        'colsample_bytree': 0.7,
        'subsample_freq':1,
        'subsample':0.7,
        'n_estimators':1600,
        'max_bin':255,
        'verbose':-1
       }
clf1 = lgb.LGBMClassifier(**params1)
rfe1 = RFECV(estimator=clf1, step=10, cv=KFold(n_splits=5, shuffle=False), 
             scoring='roc_auc', verbose=2)
rfe1.fit(X_1, y)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import suptitle
print('Optimal number of features:', rfe1.n_features_)
plt.figure(figsize=(10, 5))
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.plot(range(1, len(rfe1.grid_scores_) + 1), rfe1.grid_scores_)
plt.show()
feature_list_1 = list(X_1.columns[rfe1.ranking_ == 1])
for i in feature_list_1:
    print("'"+i+"'",end=',')

In [1]:
#rank=1  
rank1=['TransactionDT','ProductCD','P_emaildomain','R_emaildomain','nulls','month','hour','TransactionAmt',
'TransactionAmt_decimal','D5','D2','D8','D11','D15','C1','C4','C8','C10','C13','C2','V256','V13',
'card1','card2','card3','card4','card5','card6','addr1','M4','M5','M6','M2','M3','M8','M9',
'DeviceType','DeviceInfo','D6','D13','C5','C9','D7','C14','V3','V4','V5','V6','V7','V10','V11','V12','V19',
'V20','V29','V30','V34','V35','V36','V37','V38','V40','V44','V45','V47','V48','V49','V52','V53','V54','V56',
'V61','V62','V70','V73','V74','V75','V76','V78','V81','V82','V83','V87','V90','V96','V99','V126','V127','V128',
'V130','V131','V139','V143','V150','V152','V160','V165','V170','V187','V189','V202','V203','V204','V207','V208',
'V209','V212','V214','V215','V221','V222','V229','V234','V243','V257','V258','V259','V261','V262','V263','V264',
'V265','V266','V267','V268','V270','V271','V274','V276','V277','V278','V279','V280','V282','V283','V285','V291',
'V292','V294','V303','V306','V307','V308','V310','V312','V313','V314','V315','V317','V332','id_01','id_02','id_03',
'id_05','id_06','id_09','id_11','id_13','id_14','id_17','id_19','id_20','id_30','id_31','id_38','dist1','dist2',
'M4_count','M6_count','card1_count','card6_count','id_30_count','id_06_count','DeviceType_count',
'ProductCD_count','P_emaildomain_count','addr1_count','card4_dic','card6_dic','M4_dic','M5_dic','M6_dic',
'P_emaildomain_dic','R_emaildomain_dic','TransactionAmt_check','uid_count','uid2_count','uid3_count',
'card1_TransactionAmt_mean','card1_TransactionAmt_std','card2_TransactionAmt_mean','card2_TransactionAmt_std',
'card3_TransactionAmt_mean','card5_TransactionAmt_mean','card5_TransactionAmt_std','device_name','device_version',
'version_id_30','browser_id_31','version_id_31','screen_width','screen_height','M_na','uid_TransactionAmt_mean',
'uid_TransactionAmt_std','uid2_TransactionAmt_mean','uid2_TransactionAmt_std','uid3_TransactionAmt_mean',
'uid3_TransactionAmt_std','card1__dist1','card1__P_emaildomain','addr1__card1','card4__dist1','addr1__card4',
'addr1__card2','card1__card5','card5__P_emaildomain']

rank2=['V64','V80','V85','V91','V210','V216','V245','V288','V289','ProductCD_dic']

len(rank1)

217

In [6]:
feature_list={}
feature_list['1']=rank1
feature_list

{'1': ['TransactionDT',
  'ProductCD',
  'P_emaildomain',
  'R_emaildomain',
  'nulls',
  'month',
  'hour',
  'TransactionAmt',
  'TransactionAmt_decimal',
  'D5',
  'D2',
  'D8',
  'D11',
  'D15',
  'C1',
  'C4',
  'C8',
  'C10',
  'C13',
  'C2',
  'V256',
  'V13',
  'card1',
  'card2',
  'card3',
  'card4',
  'card5',
  'card6',
  'addr1',
  'M4',
  'M5',
  'M6',
  'M2',
  'M3',
  'M8',
  'M9',
  'DeviceType',
  'DeviceInfo',
  'D6',
  'D13',
  'C5',
  'C9',
  'D7',
  'C14',
  'V3',
  'V4',
  'V5',
  'V6',
  'V7',
  'V10',
  'V11',
  'V12',
  'V19',
  'V20',
  'V29',
  'V30',
  'V34',
  'V35',
  'V36',
  'V37',
  'V38',
  'V40',
  'V44',
  'V45',
  'V47',
  'V48',
  'V49',
  'V52',
  'V53',
  'V54',
  'V56',
  'V61',
  'V62',
  'V70',
  'V73',
  'V74',
  'V75',
  'V76',
  'V78',
  'V81',
  'V82',
  'V83',
  'V87',
  'V90',
  'V96',
  'V99',
  'V126',
  'V127',
  'V128',
  'V130',
  'V131',
  'V139',
  'V143',
  'V150',
  'V152',
  'V160',
  'V165',
  'V170',
  'V187',
  'V189',
  'V

In [ ]:
feature_list={}
for i in range(1,13):
    feature_list[str(i)] = list(X_1.columns[rfe1.ranking_ == i])

import csv
for i in feature_list_1:
    print("'"+i+"'",end=',')
    
with open('feature_list_1','w') as mylist:
    wr=csv.writer(mylist,quoting=csv.QUOTE_ALL)
    wr.writerow(feature_list_1)